In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import os
import pickle
import time
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from gensim.models import FastText
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Download necessary NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 1000)

print("Libraries loaded successfully!")

# Create directories for models if they don't exist
os.makedirs('models', exist_ok=True)

# Load preprocessed data
df = pd.read_csv('data/GojekAppReview_Processed.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print("\nColumn information:")
print(df.info())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Display sample data
print("\nSample data:")
print(df.head())

# Sentiment distribution
print("\nSentiment distribution:")
sentiment_counts = df['sentimenLabel'].value_counts()
print(sentiment_counts)

# Visualize sentiment distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='sentimenLabel', data=df, palette='viridis')
plt.title('Sentiment Distribution', fontsize=16)
plt.xlabel('Sentiment', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Visualize rating distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='score', data=df, palette='plasma')
plt.title('Rating Distribution', fontsize=16)
plt.xlabel('Rating', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Relationship between score and sentiment
plt.figure(figsize=(10, 6))
cross_tab = pd.crosstab(df['score'], df['sentimenLabel'])
cross_tab.plot(kind='bar', stacked=True, colormap='viridis')
plt.title('Relationship between Score and Sentiment', fontsize=16)
plt.xlabel('Score', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Sentiment', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Content length analysis
df['content_length'] = df['content'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
plt.figure(figsize=(12, 6))
sns.boxplot(x='sentimenLabel', y='content_length', data=df, palette='viridis')
plt.title('Review Length by Sentiment', fontsize=16)
plt.xlabel('Sentiment', fontsize=14)
plt.ylabel('Review Length (characters)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Create a copy of preprocessed data for model training
model_df = df.copy()

# Prepare the data for modeling with proper handling of NaN values
X = model_df['content'].fillna('')  # Replace NaN values with empty strings
y = model_df['sentimenLabel']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

# Feature Engineering Method 1: TF-IDF Vectorization
print("\nApplying TF-IDF Vectorization...")
tfidf_vectorizer = TfidfVectorizer(
    min_df=5,                # Minimum document frequency
    max_df=0.8,              # Maximum document frequency
    sublinear_tf=True,       # Apply sublinear tf scaling (1 + log(tf))
    use_idf=True,            # Apply IDF weighting
    ngram_range=(1, 2)       # Use unigrams and bigrams
)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"TF-IDF feature matrix shape: {X_train_tfidf.shape}")
print(f"Number of features: {len(tfidf_vectorizer.get_feature_names_out())}")

# Feature Engineering Method 2: Bag of Words (CountVectorizer)
print("\nApplying Bag of Words Vectorization...")
count_vectorizer = CountVectorizer(
    min_df=5,                # Minimum document frequency
    max_df=0.8,              # Maximum document frequency
    ngram_range=(1, 2)       # Use unigrams and bigrams
)

# Fit and transform the training data
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

print(f"Bag of Words feature matrix shape: {X_train_bow.shape}")
print(f"Number of features: {len(count_vectorizer.get_feature_names_out())}")

# Feature Engineering Method 3: FastText Word Embeddings
print("\nTraining FastText Word Embeddings...")

# Tokenize text for FastText
def tokenize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    return text.split()

# Prepare sentences for FastText training
tokenized_sentences = [tokenize_text(text) for text in X_train]

# Train FastText model
fasttext_model = FastText(
    sentences=tokenized_sentences,
    vector_size=100,         # Embedding dimension
    window=5,                # Context window size
    min_count=5,             # Minimum word count
    workers=4,               # Number of threads
    sg=1,                    # Skip-gram model (1) vs CBOW (0)
    epochs=10                # Number of training epochs
)

print("FastText model trained successfully")

# Function to create document vectors from FastText word embeddings
def document_vector(text, model, vector_size=100):
    words = tokenize_text(text)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

# Create document vectors for training and testing data
X_train_fasttext = np.array([document_vector(text, fasttext_model) for text in X_train])
X_test_fasttext = np.array([document_vector(text, fasttext_model) for text in X_test])

print(f"FastText document vectors shape: {X_train_fasttext.shape}")

# Prepare data for deep learning model
print("\nPreparing data for deep learning model...")
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences for consistent input length
max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

print(f"Padded sequence shape: {X_train_pad.shape}")

# Define evaluation function
def evaluate_model(y_true, y_pred, model_name):
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # Print results
    print(f"\n{model_name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    # Plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['negatif', 'netral', 'positif'],
                yticklabels=['negatif', 'netral', 'positif'])
    plt.title(f'Confusion Matrix - {model_name}', fontsize=16)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    plt.show()
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    
    return accuracy, precision, recall, f1

# Function for cross-validation
def cross_validate_model(model, X, y, cv=5, model_name=""):
    print(f"\nPerforming {cv}-fold cross-validation for {model_name}...")
    
    # Define cross-validation strategy
    kfold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    
    # Perform cross-validation
    cv_accuracy = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
    cv_precision = cross_val_score(model, X, y, cv=kfold, scoring='precision_weighted')
    cv_recall = cross_val_score(model, X, y, cv=kfold, scoring='recall_weighted')
    cv_f1 = cross_val_score(model, X, y, cv=kfold, scoring='f1_weighted')
    
    # Print results
    print(f"Cross-Validation Results for {model_name}:")
    print(f"Accuracy: {cv_accuracy.mean():.4f} (±{cv_accuracy.std():.4f})")
    print(f"Precision: {cv_precision.mean():.4f} (±{cv_precision.std():.4f})")
    print(f"Recall: {cv_recall.mean():.4f} (±{cv_recall.std():.4f})")
    print(f"F1 Score: {cv_f1.mean():.4f} (±{cv_f1.std():.4f})")
    
    return {
        'accuracy': cv_accuracy,
        'precision': cv_precision,
        'recall': cv_recall,
        'f1': cv_f1
    }

# Model 1: Multinomial Naive Bayes with TF-IDF features
print("\nTraining Multinomial Naive Bayes model with TF-IDF features...")
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_nb_tfidf = nb_tfidf.predict(X_test_tfidf)

# Evaluate the model
nb_tfidf_metrics = evaluate_model(y_test, y_pred_nb_tfidf, "Naive Bayes (TF-IDF)")

# Cross-validation
nb_tfidf_cv = cross_validate_model(MultinomialNB(), X_train_tfidf, y_train, cv=5, model_name="Naive Bayes (TF-IDF)")

# Model 2: Linear SVM with TF-IDF features
print("\nTraining Linear SVM model with TF-IDF features...")
svm_tfidf = LinearSVC(random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)

# Evaluate the model
svm_tfidf_metrics = evaluate_model(y_test, y_pred_svm_tfidf, "Linear SVM (TF-IDF)")

# Cross-validation
svm_tfidf_cv = cross_validate_model(LinearSVC(random_state=42), X_train_tfidf, y_train, cv=5, model_name="Linear SVM (TF-IDF)")

# Model 3: Random Forest with FastText features
print("\nTraining Random Forest model with FastText features...")
rf_fasttext = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fasttext.fit(X_train_fasttext, y_train)
y_pred_rf_fasttext = rf_fasttext.predict(X_test_fasttext)

# Evaluate the model
rf_fasttext_metrics = evaluate_model(y_test, y_pred_rf_fasttext, "Random Forest (FastText)")

# Cross-validation
rf_fasttext_cv = cross_validate_model(RandomForestClassifier(n_estimators=100, random_state=42), 
                                     X_train_fasttext, y_train, cv=5, 
                                     model_name="Random Forest (FastText)")

# Compare traditional ML models
models = ['Naive Bayes (TF-IDF)', 'Linear SVM (TF-IDF)', 'Random Forest (FastText)']
accuracies = [nb_tfidf_metrics[0], svm_tfidf_metrics[0], rf_fasttext_metrics[0]]
precisions = [nb_tfidf_metrics[1], svm_tfidf_metrics[1], rf_fasttext_metrics[1]]
recalls = [nb_tfidf_metrics[2], svm_tfidf_metrics[2], rf_fasttext_metrics[2]]
f1_scores = [nb_tfidf_metrics[3], svm_tfidf_metrics[3], rf_fasttext_metrics[3]]

# Plot comparison
plt.figure(figsize=(12, 8))
x = np.arange(len(models))
width = 0.2
plt.bar(x - width*1.5, accuracies, width, label='Accuracy', color='skyblue')
plt.bar(x - width/2, precisions, width, label='Precision', color='lightgreen')
plt.bar(x + width/2, recalls, width, label='Recall', color='salmon')
plt.bar(x + width*1.5, f1_scores, width, label='F1 Score', color='purple')
plt.ylim(0, 1.0)
plt.xlabel('Models', fontsize=14)
plt.ylabel('Score', fontsize=14)
plt.title('Traditional ML Models Performance Comparison', fontsize=16)
plt.xticks(x, models, rotation=45, ha='right')
plt.legend(loc='lower right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Save the best traditional model
best_traditional_model = svm_tfidf  # Assuming SVM performs best, adjust based on actual results
with open('models/svm_tfidf_model.pkl', 'wb') as f:
    pickle.dump(best_traditional_model, f)
with open('models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print("Best traditional model saved!")

# Prepare label encoding for deep learning
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Get number of classes
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print(f"Classes: {label_encoder.classes_}")

# Create embeddings matrix for pre-trained weights
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Fill embedding matrix with FastText vectors
for word, i in tokenizer.word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[i] = fasttext_model.wv[word]

# Build Bidirectional LSTM model with FastText embeddings
print("\nBuilding Bidirectional LSTM model...")
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, 
                   weights=[embedding_matrix], 
                   input_length=max_length, 
                   trainable=False))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train model with early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train_pad, y_train_encoded,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[early_stop],
                    verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test_pad, y_test_encoded)
print(f"\nBiLSTM Model Test Accuracy: {accuracy:.4f}")

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.tight_layout()
plt.show()

# Make predictions
y_pred_probs = model.predict(X_test_pad)
y_pred_encoded = np.argmax(y_pred_probs, axis=1)
y_pred_dl = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate the deep learning model
dl_metrics = evaluate_model(y_test, y_pred_dl, "BiLSTM with FastText Embeddings")

# Save the deep learning model
model.save('models/bilstm_model.keras')
with open('models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Deep learning model saved!")

# Word cloud visualization for each sentiment
def generate_wordcloud(text, title):
    wordcloud = WordCloud(
        width=800, 
        height=400, 
        background_color='white', 
        colormap='viridis',
        max_words=100,
        contour_width=3,
        contour_color='steelblue'
    ).generate(text)
    
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Create wordclouds for each sentiment
for sentiment in ['positif', 'netral', 'negatif']:
    # Filter out NaN values and convert all values to strings
    filtered_content = df[df['sentimenLabel'] == sentiment]['content'].dropna().astype(str)
    sentiment_texts = ' '.join(filtered_content)
    generate_wordcloud(sentiment_texts, f'Word Cloud - {sentiment.capitalize()} Reviews')

# N-gram analysis
def get_top_n_words(corpus, n_gram_range, n=10):
    vec = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords.words('indonesian')).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

# Get top unigrams for each sentiment
for sentiment in ['positif', 'netral', 'negatif']:
    # Filter out NaN values before processing
    sentiment_texts = df[df['sentimenLabel'] == sentiment]['content'].dropna().astype(str)
    top_words = get_top_n_words(sentiment_texts, (1, 1), 15)
    
    words = [word[0] for word in top_words]
    counts = [word[1] for word in top_words]
    
    plt.figure(figsize=(12, 6))
    plt.bar(words, counts, color='skyblue')
    plt.title(f'Top 15 Words in {sentiment.capitalize()} Reviews', fontsize=16)
    plt.xlabel('Words', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Length analysis by sentiment
plt.figure(figsize=(10, 6))
for sentiment in ['positif', 'netral', 'negatif']:
    sns.kdeplot(df[df['sentimenLabel'] == sentiment]['content_length'], label=sentiment)
plt.title('Review Length Distribution by Sentiment', fontsize=16)
plt.xlabel('Review Length (characters)', fontsize=14)
plt.ylabel('Density', fontsize=14)
plt.legend(title='Sentiment')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# Time-based analysis
df['date'] = pd.to_datetime(df['at']).dt.date
sentiment_by_date = df.groupby(['date', 'sentimenLabel']).size().unstack().fillna(0)

# Calculate sentiment ratio over time
sentiment_ratio = sentiment_by_date.div(sentiment_by_date.sum(axis=1), axis=0)

# Plot sentiment trends over time
plt.figure(figsize=(14, 7))
sentiment_ratio.plot(kind='line', linewidth=2)
plt.title('Sentiment Ratio Trends Over Time', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Proportion of Sentiments', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()

Libraries loaded successfully!
Dataset shape: (225002, 6)

Column information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225002 entries, 0 to 225001
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   userName       225002 non-null  object
 1   content        225000 non-null  object
 2   score          225002 non-null  int64 
 3   at             225002 non-null  object
 4   appVersion     225002 non-null  object
 5   sentimenLabel  225002 non-null  object
dtypes: int64(1), object(5)
memory usage: 10.3+ MB
None

Missing values:
userName         0
content          2
score            0
at               0
appVersion       0
sentimenLabel    0
dtype: int64

Sample data:
                  userName                                            content  score                   at appVersion sentimenLabel
0                Yuga Edit                            akun gopay saya di blok      1  2022-01-21 10:52:12      4.9

KeyboardInterrupt: 